In [1]:
import pandas, numpy, os, re

In [2]:
article_directory = './hw01-data-raw/News Articles'
article_list = os.listdir(article_directory) # create list of files
article_list[:5]

['1101243072871.txt',
 '1101163341619.txt',
 '1101163941020.txt',
 '1101163268260.txt',
 '1101163374885.txt']

In [39]:
keywords = { 
    'mad cow': ['mad cow', 'prion', 'creutzfeldt-jakob', 'spongiform', 'encephalopath'], 
    'laboratory': ['lab'],
    'whistleblower': ['whistle'],
    'key names': ['Ryker', 'Sanderson']
}
keywords.keys()

dict_keys(['mad cow', 'laboratory', 'whistleblower', 'key names'])

regular expression from https://stackoverflow.com/questions/17268958/finding-occurrences-of-a-word-in-a-string-in-python-3

In [43]:
article_data = []
# create dataframe from list of dictionaries
# each dictionary: {filename: filename.txt, title: "", author: {author name}, date: {date posted}, ...}

for file_name in article_list:
    # open the file
    article = open(f'{article_directory}/{file_name}', encoding='ISO 8859-1')
    article_text = article.read()
    # replace weird characters and newlines with single space, convert to lower case
    # article_text = article_text.replace('\x97', ' ').replace('\xa0', ' ').replace('\n', ' ')
    # article_text = re.sub(' +', ' ', article_text).lower()

    # empty list of article keywords to populate
    article_keywords = []
    article_dict = { 'filename': file_name}
    # get other metadata for article_dict from Emi's code (article title, author, date published)
    article_text = article_text.replace('\x97', ' ').replace('\xa0', ' ').replace('\n', '#')
    article_text = re.sub(' +', ' ', article_text).lower()
    article_text = re.sub('(#)+', '#', article_text)
    article_text = article_text[1:]
    article_pieces =  article_text.split('#')
    # print(article_pieces)
    title = article_pieces[0]
    author = False
    date = False
    if "story by:" in article_pieces[1]:
        author = article_pieces[1][10:]
        date = article_pieces[2][23:]
    elif article_pieces[0] == " ":
        date = article_pieces[1].strip()[23:]
    else:
        date = article_pieces[2].strip()[23:]
    article_dict['title'] = title
    article_dict['date'] = date
    article_dict['author'] = author
    # tally up occurrences of each keyword category
    tally = 0
    for keyword in keywords.keys():
        article_dict[keyword] = 0
        contains_keyword = False
        for synonym in keywords[keyword]:
            num_found = sum(1 for _ in re.finditer(r'%s' % re.escape(synonym.lower()), article_text))
            article_dict[keyword] += num_found
            tally+=num_found
    if tally>0:
        article_data.append(article_dict)
article_data

[{'filename': '1101243072871.txt',
  'title': 'entertainment briefs',
  'date': '4/23/2003',
  'author': False,
  'mad cow': 0,
  'laboratory': 1,
  'whistleblower': 0,
  'key names': 0},
 {'filename': '1101163341619.txt',
  'title': "alderwood high school honor students who go 'above and beyond' recognized",
  'date': '5/6/2002',
  'author': 'jamie smart',
  'mad cow': 0,
  'laboratory': 2,
  'whistleblower': 0,
  'key names': 0},
 {'filename': '1101243294357.txt',
  'title': 'local dentist honored for work with low-income children',
  'date': '2/10/2004',
  'author': 'melissa browning',
  'mad cow': 0,
  'laboratory': 1,
  'whistleblower': 0,
  'key names': 0},
 {'filename': '1101162453186.txt',
  'title': "hefty increase in alderwood property tax rate ok'd",
  'date': '11/9/2004',
  'author': 'john panni',
  'mad cow': 0,
  'laboratory': 1,
  'whistleblower': 0,
  'key names': 0},
 {'filename': '1101243533827.txt',
  'title': 'recall expands to eight states, guam',
  'date': '12/29/

In [44]:
article_df = pandas.DataFrame(article_data)
article_df.sort_values(by='key names', ascending=False)

,filename,title,date,author,mad cow,laboratory,whistleblower,key names
54,1101163595600.txt,alderwood lab breakthrough in mad cow disease,3/2/2004,john panni,15,4,0,3
140,1101163840166.txt,synthetic prion causes neurological disease in...,1/20/2004,mel sacher,12,1,0,2
113,1101163977242.txt,fda cancels investigation,6/19/2004,mel sacher,0,3,0,2
282,1101163655870.txt,seniors dominate 4.00 gpa list at alderwood high,2/19/2003,False,0,0,0,1
148,1101163018599.txt,testimonials for new restaurant,9/10/2003,False,0,0,0,1
...,...,...,...,...,...,...,...,...
101,1101162805067.txt,sunshine days celebration plans coming together,9/10/2004,jamie smart,0,1,0,0
102,1101163794106.txt,alderwood building projects in december total ...,1/27/2003,ellie olmsen,0,3,0,0
103,1101243444577.txt,whistle blowers say mad cow disease testing mu...,1/15/2002,ellie olmsen,5,0,4,0
104,1101242876714.txt,prosser memorial new computer system will help...,7/6/2003,False,0,2,0,0
